# カタカナ15文字を自動識別するモデルの構築
# 課題投稿用Notebook
* これは課題投稿用ファイルです。
* このsubmit_katakana.ipynbファイルを編集し、学習済みモデルなどの必要ファイルを同じフォルダにおき、そのフォルダをzipしたものを投稿してください。

In [1]:
import numpy as np
import glob
import os,sys
import util
import pickle
#import matplotlib.pyplot as plt
from common.layers             import *

def makedataset():
    """
    データセットをつくる関数です。
    自由に編集してください。
    """
    
    # 次の行は変更しないこと
    test_data= util.loaddata()
    
    # 以下は自由に編集しても構いません
    # 必要な前処理をここに記述してください  
    
    # 正規化
    test_data = test_data / test_data.max()
    test_data = test_data.astype('float32')

    # 配列形式変更
    #test_data = test_data.reshape(-1, 28*28)
    test_data = test_data.reshape(-1, 1, 28, 28) # CNN対応

    return test_data


def func_predict(test_data, test_label):
    """
    予測する関数
    data : 画像データ
    return loss, accuracy
    引数とreturn以外は、自由に編集してください    
    """
    
    # 以下を自由に編集してください
    with open("katakana_model.pickle", "rb") as f:
        load_prms = pickle.load(f)
        
    #print(load_prms.keys())

    # CNN
    snet = MultiLayerConvNet(input_dim=(1, 28, 28), cnn_layers_list=[
        {'Conv111':Conv2D(filter_num=30,filter_size=5,pad=0,stride=1)},
        {'Pool111':Pool2D(pool_size=2,pad=0,stride=2)},
        {'Conv121':Conv2D(filter_num=40,filter_size=5,pad=0,stride=1)},
        {'Pool121':Pool2D(pool_size=2,pad=0,stride=2)}],                    
        hidden_size_list=[120,120],
        output_size=15, weight_init_std=0.01, use_batchnorm=True, weight_decay_lambda=0.00,
        use_dropout=False, dropout_ration=0.0)


    # パラメーター設定
    for layer in snet.layers.keys():
        id = layer[-3:]
        if('Conv' in layer or 'Affine' in layer):
            snet.layers[layer].W = load_prms['W'+id]
            snet.layers[layer].b = load_prms['b'+id]
        elif('BatchNorm' in layer):
            snet.layers[layer].moving_mean = load_prms[layer + '_mv_mu']
            snet.layers[layer].moving_var = load_prms[layer + '_mv_var']
            snet.layers[layer].gamma = load_prms['gamma' + id]
            snet.layers[layer].beta = load_prms['beta' + id]
    
    accuracy = snet.accuracy(test_data, test_label, train_flg=False)
    loss  = snet.loss(test_data, test_label, train_flg=False)
    
    # 不正解データ出力
    ng_dic, ng_img_list, miss_read_lbl_list, correct_lbl_list = snet.get_miss_read_data(test_data, test_label)

    print('\n▼ Miss Read Count ：', len(correct_lbl_list))
    for k in sorted(ng_dic,key=ng_dic.get, reverse=True):
        print((k + ' -').ljust(5,'-') + '> ',ng_dic[k])

    print('\n▼ Miss Read Image：')
    for idx,label in enumerate(correct_lbl_list):
        print('miss_read：',miss_read_lbl_list[idx],' ---> correct：',label)
    
    return loss, accuracy # 編集不可


def main():
    """
    編集しないでください。
    """
    # テスト用データをつくる
    test_data = makedataset()

    # 予測し精度を算出する
    util.accuracy(func_predict, test_data)
    
    return


if __name__=="__main__":
    main()


▼minibatch_num= 0
----------------------------------------------------
self.use_batchnorm <-------------------- True
self.weight_d_lmda <-------------------- 0.0
self.use_dropout <---------------------- False
self.dropout_ration <------------------- 0.0
self.hidden_size_list <----------------- [120, 120]
----------------------------------------------------
out_size <------------------------------ [28, 24, 12, 8, 4]
out_pixel <----------------------------- 640
----------------------------------------------------
Conv111 <------------------------------- W111: (30, 1, 5, 5) b111: (30,)
BatchNorm111 <-------------------------- gamma111: (30,) beta111: (30,)
ReLU111 <-------------------------------
Pool111 <-------------------------------
Conv121 <------------------------------- W121: (40, 30, 5, 5) b121: (40,)
BatchNorm121 <-------------------------- gamma121: (40,) beta121: (40,)
ReLU121 <-------------------------------
Pool121 <-------------------------------
Affine211 <----------------


▼ Miss Read Count ： 0

▼ Miss Read Image：
loss ---->  3.2059975819600335e-05 
accuracy ---->  1.0

▼結果
Test loss: 0.0001151588028539234
Test accuracy: 1.0
